# CS550 Project Phase 1

For the Checkpoint:
wallet, transaction definition, block definition, proof of work,proof of memory, proof of space (extra credit 10 points)


To implement the centralized architecture with the six components mentioned, it will need to create separate processes for
each component that can communicate with each other using network sockets. 
Here's a high-level overview of how to implement each component:

1.Wallet:

Implement functionality to create wallets, send transactions, and view balance.
Establish a network socket connection with the other components to send and receive information.

2.Blockchain:

Implement functionality to store blocks and retrieve blocks.
Implement functionality to view address balance.
Establish a network socket connection with other components to receive block and transaction information.

3.Pool:

Implement functionality to store unconfirmed transactions.
Establish a network socket connection with the Validator component to receive and send transaction information.

4.Metronome:

Implement functionality to keep track of validators and netspace.
Ensure blocks are generated at consistent intervals.
Establish network socket connections with other components to exchange information related to validators and blocks.

5.Validator + PoW/PoM/PoS:

Implement functionality to perform Proof of Work (PoW), Proof of Stake (PoS), or Proof of Misbehaviour (PoM) based on the requirements.
Retrieve transactions from the pool and verify their validity.
Communicate with other validators to verify blocks.
Send the final validated block to the Blockchain component.
Establish network socket connections with the Pool, Blockchain, and other Validator components.

6.Monitor:

Implement functionality to monitor unconfirmed transactions and blocks stored in the blockchain.
Keep track of the number of validators and the amount of netspace.
Establish network socket connections with other components to receive and display relevant information.

In the program "dsc,"  it will create separate processes for each component and establish network socket connections between them to
enable communication. Each process will handle the specific functionality of its component and interact with other components as 
required.

Implementing the entire centralized architecture with these components requires a significant amount of work and expertise 
in networking and concurrent programming. You will need to design appropriate protocols for communication, handle 
synchronization and concurrency challenges, and ensure the components work together seamlessly.

1. •Create wallet
–Use SHA256 to create public/private keys of 256bit length, store in wallet.cfg in Base58 encoding
–Wallet.cfg
•public_key: Base58
•private_key: Base58

In [1]:
import hashlib
import base58
import os

def generate_wallet():
    # Generate a random 256-bit private key
    private_key = os.urandom(32)

    # Calculate the corresponding public key using SHA256
    public_key = hashlib.sha256(private_key).digest()

    # Encode the keys in Base58 format
    encoded_private_key = base58.b58encode(private_key)
    encoded_public_key = base58.b58encode(public_key)

    # Create a dictionary to store the keys
    wallet = {
        'public_key': encoded_public_key,
        'private_key': encoded_private_key
    }

    return wallet

# Generate a wallet
wallet = generate_wallet()
# save the key to the wallet.cfg
with open('wallet.cfg', 'w') as file:
    file.write(f"public_key: {wallet['public_key'].decode()}\n")
    file.write(f"private_key: {wallet['private_key'].decode()}\n")
print("Wallet created and keys stored in wallet.cfg.")



Wallet created and keys stored in wallet.cfg.


The code of create a wallet and store the public and private keys in a configuration file called wallet.cfg, using SHA256 for key generation and Base58 encoding is listed as below.Save the code in a Python file (e.g., wallet.py) and run it. It will generate a wallet, encode the keys in Base58, and store them in the wallet.cfg file.

************************************************************************
To implement the Wallet component with functionality to create wallets, send transactions, and view balances, as well as establish a network socket connection with other components, you can follow these steps:

Import the necessary libraries for socket communication. For example, in Python, you can use the socket module.

Create a socket to establish a connection with the desired component. You will need to know the IP address and port number of the component you want to connect to.

Implement functions to create wallets, send transactions, and view balances. These functions can interact with the user or take inputs programmatically.

Use the socket to send requests or messages to the desired component. You can define a protocol or message format to communicate the specific actions or data.

Receive responses from the component using the socket and process them accordingly. Again, you can define a protocol or message format to handle the responses.

Wrap the functionality of the Wallet component in a loop or user interface so that users can interact with it continuously.

*************************************************************************
To implement the send_transaction() function, you can follow these steps:

Gather the necessary inputs for the transaction, such as the sender's public address, recipient's public address, value, timestamp, transaction ID, and signature. You can obtain these inputs from the user or generate them programmatically.

Create a Transaction object using the gathered inputs.

Serialize the Transaction object into bytes using the serialize() method of the Transaction class.

Establish a network socket connection with the desired component to send the transaction. You will need to know the IP address and port number of the component.

Send the serialized transaction data over the socket connection using the send() method.

Receive a response or acknowledgment from the component to confirm the successful receipt or processing of the transaction, if applicable.

Here's an example implementation of the send_transaction() function based on the provided specifications:

In [6]:
import hashlib
import base58
import os
import socket
import struct
from transaction import Transaction

class Transaction:
    def __init__(self, sender_address, recipient_address, value, timestamp, transaction_id, signature):
        self.sender_address = sender_address
        self.recipient_address = recipient_address
        self.value = value
        self.timestamp = timestamp
        self.transaction_id = transaction_id
        self.signature = signature
    
    def serialize(self):
        # Serialize the transaction object into bytes
        transaction_data = struct.pack('32s32sdbl16s32s',
                                       self.sender_address,
                                       self.recipient_address,
                                       self.value,
                                       self.timestamp,
                                       self.transaction_id,
                                       self.signature)
        return transaction_data
    
    @classmethod
    def deserialize(cls, data):
        # Deserialize the bytes into a transaction object
        transaction_fields = struct.unpack('32s32sdbl16s32s', data)
        transaction = cls(*transaction_fields)
        return transaction

class Block:
    def __init__(self, block_size, version, previous_block_hash, block_id, timestamp, difficulty_target, nonce, transactions):
        self.block_size = block_size
        self.version = version
        self.previous_block_hash = previous_block_hash
        self.block_id = block_id
        self.timestamp = timestamp
        self.difficulty_target = difficulty_target
        self.nonce = nonce
        self.transactions = transactions
    
    def serialize(self):
        # Serialize the block object into bytes
        block_header = struct.pack('I56s', self.version, self.previous_block_hash)
        block_data = struct.pack('I', self.block_size) + block_header
        block_data += struct.pack('IqHQB64s', self.block_id, self.timestamp, self.difficulty_target, self.nonce, 0, b'')
        
        for transaction in self.transactions:
            block_data += transaction.serialize()
        
        return block_data
    
    @classmethod
    def deserialize(cls, data):
        # Deserialize the bytes into a block object
        block_size = struct.unpack('I', data[:4])[0]
        version, previous_block_hash = struct.unpack('I56s', data[4:60])
        block_id, timestamp, difficulty_target, nonce = struct.unpack('IqHQB', data[60:81])
        
        transactions_data = data[81:]
        transactions = []
        transaction_size = 128
        for i in range(block_size):
            transaction_data = transactions_data[i * transaction_size: (i + 1) * transaction_size]
            transaction = Transaction.deserialize(transaction_data)
            transactions.append(transaction)
        
        block = cls(block_size, version, previous_block_hash, block_id, timestamp, difficulty_target, nonce, transactions)
        return block




def create_wallet():
    # Implementation to create a wallet
     # Generate a random 256-bit private key
    private_key = os.urandom(32)

    # Calculate the corresponding public key using SHA256
    public_key = hashlib.sha256(private_key).digest()

    # Encode the keys in Base58 format
    encoded_private_key = base58.b58encode(private_key)
    encoded_public_key = base58.b58encode(public_key)

    # Create a dictionary to store the keys
    wallet = {
        'public_key': encoded_public_key,
        'private_key': encoded_private_key
    }

    return wallet


def send_transaction():
    # Gather inputs for the transaction
    sender_address = input("Enter sender's public address: ")
    recipient_address = input("Enter recipient's public address: ")
    value = float(input("Enter value: "))
    timestamp = int(input("Enter timestamp: "))
    transaction_id = input("Enter transaction ID: ")
    signature = input("Enter signature: ")

    # Create a Transaction object
    transaction = Transaction(sender_address, recipient_address, value, timestamp, transaction_id, signature)

    # Serialize the Transaction object into bytes
    transaction_data = transaction.serialize()

    # Establish a socket connection with the desired component
    socket_connection = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socket_connection.connect(('component_ip', component_port))

    # Send the transaction data over the socket connection
    socket_connection.send(transaction_data)

    # Receive a response or acknowledgment from the component
    response = socket_connection.recv(1024)
    # Process the response here

    socket_connection.close()

def view_balance():
    # Implementation to view balance
    if os.path.isfile(WALLET_FILE):
        with open(WALLET_FILE, 'r') as file:
            for line in file:
                if line.startswith('balance'):
                    balance = int(line.split(':')[1].strip())
                    print(f"Balance: {balance} coins")
                    return
    else:
        print("Wallet does not exist.")







while True:
    # User interaction or programmatically trigger actions
    action = input("Enter action (create/send/view): ")

    if action == "create":
        create_wallet()
        # Store the keys in a configuration file
        # save the key to the wallet.cfg
        with open('wallet.cfg', 'w') as file:
            file.write(f"public_key: {wallet['public_key'].decode()}\n")
            file.write(f"private_key: {wallet['private_key'].decode()}\n")
        print("Wallet created and keys stored in wallet.cfg.")

        # Send a message to the component to create a wallet
        wallet_socket.send(b'CREATE_WALLET')

    elif action == "send":
        send_transaction()
        # Send a message to the component to send a transaction
        wallet_socket.send(b'SEND_TRANSACTION')

    elif action == "view":
        view_balance()
        # Send a message to the component to view balance
        wallet_socket.send(b'VIEW_BALANCE')

    # Receive and process the response from the component
    response = wallet_socket.recv(1024)
    # Process the response here

wallet_socket.close()

Enter action (create/send/view): create
Wallet created and keys stored in wallet.cfg.


OSError: [WinError 10057] 由于套接字没有连接并且(当使用一个 sendto 调用发送数据报套接字时)没有提供地址，发送或接收数据的请求没有被接受。

2. Operations
–Create wallet (if exists, abort)
–Print public key
–Print private key
–Get balance
-Send X coins to Y address


To perform the operations you mentioned on the wallet, you'll need to modify the previous code and add functions for each operation. Here's an updated version of the code that includes the additional operations:

In [4]:
import hashlib
import base58
from ecdsa import SigningKey, SECP256k1

def generate_keys():
    # Generate a private key
    private_key = SigningKey.generate(curve=SECP256k1)

    # Derive the corresponding public key
    public_key = private_key.get_verifying_key()

    # Convert the keys to hex format
    private_key_hex = private_key.to_string().hex()
    public_key_hex = public_key.to_string().hex()

    # Hash the public key using SHA-256
    hashed_public_key = hashlib.sha256(bytes.fromhex(public_key_hex)).digest()

    # Encode the keys as base58 strings
    encoded_private_key = base58.b58encode(bytes.fromhex(private_key_hex))
    encoded_public_key = base58.b58encode(hashed_public_key)

    return encoded_private_key.decode(), encoded_public_key.decode()

# Example usage
private_key, public_key = generate_keys()

print("Private Key:", private_key)
print("Public Key:", public_key)

Private Key: 5trk6T3gHgWQXJVB2eYLqarrWmPMy1VpezFcutcdPie5
Public Key: 2fFZ4YG1cTsAPykF3MA33KiFrzfgsYmJrzXWMmA17mJx


3. Pool, Blockchain and Metronome
- For Checkpoint #1, create skeleton codes for Pool and Blockchain
- The Pool will receive transaction and send dummy acknowledgement to the 
wallet
- Blockchain will receive getBalance request and reply with dummy balance 
message
- Blockchain will send a random 24 byte hash to the validator.
- The metronome needs to send the difficulty level to the validator, for now set 
the value to 30. (30 bits)


Here's a skeleton code for the Pool and Blockchain components.

In this skeleton code, the Pool class represents a transaction pool. It has an __init__ method to initialize the transactions list, a receive_transaction method to receive transactions from the wallet, and a send_acknowledgement method to send a dummy acknowledgement to the wallet.

In [6]:
class Pool:
    def __init__(self):
        self.transactions = []

    def receive_transaction(self, transaction):
        # Add the transaction to the pool
        self.transactions.append(transaction)

        # Send dummy acknowledgement to the wallet
        self.send_acknowledgement()

    def send_acknowledgement(self):
        # Your implementation to send an acknowledgement to the wallet
        # This could be a simple message or a response code
        print("Acknowledgement sent to wallet")


# Usage:
pool = Pool()
wallet_transaction = "Transaction data"
pool.receive_transaction(wallet_transaction)

Acknowledgement sent to wallet


In this skeleton code, the Blockchain class represents the blockchain component. 
It has a get_balance method that takes a public key as input and retrieves the balance for that key. 
It then sends a dummy balance message using the send_balance_message method. 
The send_hash_to_validator method has been added to the Blockchain class. 
It generates a random 24-byte hash using random.randint and hashlib.sha256. 
It then sends the hash to the validator using the appropriate implementation

In [ ]:
a Metronome class has been added to represent the component responsible for sending the difficulty level to the validator. The send_difficulty_level method in the Metronome class is used to send the difficulty level to the Blockchain component's receive_difficulty_level method.

In [8]:
import random
import hashlib

class Blockchain:
    def get_balance(self, public_key):
        # Your implementation to retrieve the balance for the given public key
        # This could involve querying the blockchain or a wallet service

        # Dummy balance message
        balance_message = f"Balance for public key {public_key}: 100 coins"

        # Reply with the balance message
        self.send_balance_message(balance_message)

    def send_balance_message(self, message):
        # Your implementation to send the balance message
        print("Balance message sent:", message)

    def receive_difficulty_level(self, difficulty_level):
        # Your implementation to receive the difficulty level from the metronome
        print("Received difficulty level:", difficulty_level)

    def send_hash_to_validator(self):
        # Generate a random 24-byte hash
        random_hash = hashlib.sha256(bytes(random.randint(0, 255) for _ in range(24))).digest()

        # Your implementation to send the hash to the validator
        print("Hash sent to validator:", random_hash.hex())


class Metronome:
    def send_difficulty_level(self, difficulty_level):
        # Your implementation to send the difficulty level to the blockchain
        blockchain.receive_difficulty_level(difficulty_level)


# Usage:
blockchain = Blockchain()
metronome = Metronome()

public_key = "Public key"
blockchain.get_balance(public_key)
blockchain.send_hash_to_validator()

difficulty_level = 30
metronome.send_difficulty_level(difficulty_level)

Balance message sent: Balance for public key Public key: 100 coins
Hash sent to validator: 704a90e31ecc65abac987cf51046550be5c583d65c5c968c8959050bc839759f
Received difficulty level: 30


the Validator component receiving a hash from the blockchain and attempting to generate a hash using BLAKE3. It checks if the prefix of the generated hash matches the received hash, and if so, sends a success message to the console. 
It also includes placeholders for implementing Proof of Work, Proof of Memory, and Proof of Space.

In [ ]:
import random
import hashlib
import time

class Blockchain:
    def __init__(self, validator):
        self.validator = validator

    def get_balance(self, public_key):
        # Your implementation to retrieve the balance for the given public key
        # This could involve querying the blockchain or a wallet service

        # Dummy balance message
        balance_message = f"Balance for public key {public_key}: 100 coins"

        # Reply with the balance message
        self.send_balance_message(balance_message)

    def send_balance_message(self, message):
        # Your implementation to send the balance message
        print("Balance message sent:", message)

    def receive_hash_from_blockchain(self, received_hash):
        # Pass the received hash to the validator
        self.validator.validate_hash(received_hash)


class Validator:
    def validate_hash(self, received_hash):
        # Set the prefix length
        prefix_length = 30

        # Set the time limit in seconds
        time_limit = 6

        # Generate a hash and check if the prefix matches within the time limit
        start_time = time.time()
        nonce = -1
        generated_hash = None

        while time.time() - start_time <= time_limit:
            nonce += 1
            generated_hash = self.generate_hash(nonce)

            if generated_hash[:prefix_length] == received_hash[:prefix_length]:
                break

        if generated_hash[:prefix_length] == received_hash[:prefix_length]:
            print("Success! Prefix matched within the time limit.")
            print("Nonce:", nonce)
        else:
            print("Prefix not matched within the time limit.")
            hashes_per_second = nonce / time_limit
            print("Hashes per second:", hashes_per_second)

    def generate_hash(self, nonce):
        # Generate a hash using the nonce
        nonce_bytes = nonce.to_bytes(8, 'big')
        generated_hash = hashlib.sha256(nonce_bytes).digest()
        return generated_hash.hex()


# Usage:
validator = Validator()
blockchain = Blockchain(validator)

public_key = "Public key"
blockchain.get_balance(public_key)

received_hash = "Received Hash"
blockchain.receive_hash_from_blockchain(received_hash)

Proof of memory
You need to generate a min. 1 Gb file in memory (not in disk) which contains 
hashes. 
Sort them using a sorting algorithm, that sorts in logarithmic time.
Then search the file and try to match the prefix with the hash received from the 
Blockchain. 
The success message needs to contain the NONCE from which the hash was 
generated. Or -1 in case of fail.
The file size needs to be variable.


In [ ]:
In this code, the HashFile class generates a file in memory by repeatedly generating hashes with corresponding nonces. The generate_hash method generates a hash using a nonce, and the generate_file method populates the hashes list with tuples containing the hash and nonce.

The sort_file method sorts the hashes list using a logarithmic time sorting algorithm, which in this case is the default sorting algorithm in Python (list.sort). This algorithm has an average time complexity of O(n log n).

The search_hash_prefix method searches for a given prefix in the hashes list using the bisect_left method from the bisect module. It checks if the prefix matches the hash prefix at the found index and prints the success message with the corresponding nonce if a match is found.

In [12]:
import hashlib
import random
import bisect


class HashFile:
    def __init__(self, file_size):
        self.file_size = file_size
        self.hashes = []

    def generate_file(self):
        while self.get_file_size() < self.file_size:
            nonce = random.randint(0, 2**32)  # Generate a nonce
            hash_value = self.generate_hash(nonce)
            self.hashes.append((hash_value, nonce))

    def generate_hash(self, nonce):
        # Generate a hash using the nonce
        nonce_bytes = nonce.to_bytes(8, 'big')
        hash_value = hashlib.sha256(nonce_bytes).digest()
        return hash_value.hex()

    def sort_file(self):
        self.hashes.sort(key=lambda x: x[0])

    def search_hash_prefix(self, prefix):
        prefix_length = len(prefix)
        hash_prefix = prefix.lower()

        index = bisect.bisect_left(self.hashes, (hash_prefix,), key=lambda x: x[0][:prefix_length])
        print(index)
        print(self.hases)
        if index < len(self.hashes) and self.hashes[index][0][:prefix_length] == hash_prefix:
            print("Success! Prefix matched.")
            print("Nonce:", self.hashes[index][1])
        else:
            print("Prefix not found.")

    def get_file_size(self):
        # Calculate the current size of the file in memory
        # Each hash is assumed to be 64 bytes (SHA-256 hash size)
        return len(self.hashes) * 64


# Usage:
file_size = 1024 * 1024 * 1024  # 1 GB
hash_file = HashFile(file_size)

hash_file.generate_file()
hash_file.sort_file()

prefix_to_search = "0123"  # Prefix to search for

hash_file.search_hash_prefix(prefix_to_search)

TypeError: '<' not supported between instances of 'str' and 'tuple'

Proof of Space
● It is similar to Proof of memory but the difference is that you need to generate 
the file of hashes before you receive the hash from the blockchain. 
● Store them on disk, and then search the disk for the prefix. 
● If successful send a success message to the blockchain, and a fail message if 
failed. 
● The success message needs to contain the NONCE from which the hash was 
generated. Or -1 in case of fail

In [ ]:
In this code, the HashFile class is responsible for generating a file of hashes on disk and searching for a given prefix in the file. The generate_file method generates hashes with corresponding nonces and writes them to a file specified by file_path.

The search_hash_prefix method reads the file line by line and checks if the prefix matches the hash prefix. If a match is found, it prints the success message with the corresponding nonce. If no match is found, it prints a failure message.

Please make sure to provide a valid file path for file_path and adjust the file_size and prefix_to_search variables according to your needs.

In [ ]:
import hashlib
import random


class HashFile:
    def __init__(self, file_path):
        self.file_path = file_path

    def generate_file(self, file_size):
        with open(self.file_path, 'wb') as file:
            while self.get_file_size(file) < file_size:
                nonce = random.randint(0, 2**32)  # Generate a nonce
                hash_value = self.generate_hash(nonce)
                file.write(hash_value.encode() + b'\n')

    def generate_hash(self, nonce):
        # Generate a hash using the nonce
        nonce_bytes = nonce.to_bytes(8, 'big')
        hash_value = hashlib.sha256(nonce_bytes).hexdigest()
        return hash_value

    def search_hash_prefix(self, prefix):
        prefix_length = len(prefix)
        hash_prefix = prefix.lower()

        with open(self.file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line[:prefix_length] == hash_prefix:
                    print("Success! Prefix matched.")
                    nonce = int(line[prefix_length:], 16)
                    print("Nonce:", nonce)
                    return

        print("Prefix not found.")

    def get_file_size(self, file):
        # Calculate the current size of the file on disk
        file.seek(0, 2)  # Seek to the end of the file
        file_size = file.tell()
        file.seek(0)  # Reset the file pointer to the beginning
        return file_size


# Usage:
file_path = "hashes.txt"
file_size = 1024 * 1024 * 1024  # 1 GB
hash_file = HashFile(file_path)

hash_file.generate_file(file_size)

prefix_to_search = "0123"  # Prefix to search for

hash_file.search_hash_prefix(prefix_to_search)